# SIGAA Pentest - Setup e Execução
Testes de autenticação autorizados no SIGAA.

**setup:**
```bash
# Bash (Linux/macOS)
python3 -m venv venv
source venv/bin/activate
pip install -r requirements.txt

# PowerShell (Windows)
python -m venv venv; venv\Scripts\Activate.ps1; pip install -r requirements.txt
```

**arquivo `.env` na raiz com suas credenciais:**
```
SIGAA_USER=seu_usuario
SIGAA_PASS=sua_senha
```

In [2]:
import os
import sys
from rich import print as rprint
from rich.console import Console
from rich.table import Table
from dotenv import load_dotenv

# Carregar variáveis do .env
load_dotenv()

console = Console()

rprint("[bold green]✅ Validação do Ambiente[/bold green]")

env_table = Table(title="Ambiente")
env_table.add_column("Parâmetro")
env_table.add_column("Valor")
env_table.add_row("Python", f"{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}")
env_table.add_row("Executável", sys.executable)
env_table.add_row("Diretório", os.getcwd())
env_table.add_row("Credenciais", "Carregadas" if os.getenv("SIGAA_USER") else "Não encontradas")
console.print(env_table)

if not os.getenv("SIGAA_USER"):
    rprint("[red]AVISO:[/red] Crie um arquivo .env com SIGAA_USER e SIGAA_PASS")

✅ Validação do Ambiente

                                                     Ambiente                                                      
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Parâmetro   ┃ Valor                                                                                             ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Python      │ 3.11.9                                                                                            │
│ Executável  │ z:\projects\eng-sistemas-distribuidos-ufpb\eng-sistemas-distribuidos-ufpb\venv\Scripts\python.exe │
│ Diretório   │ z:\projects\eng-sistemas-distribuidos-ufpb\eng-sistemas-distribuidos-ufpb                         │
│ Credenciais │ Carregadas                                                                                        │
└─────────────┴───────────────────────────────────────────────────────────────────────────────────────────────────┘

In [4]:
# Análise de transporte e TLS
import requests
import re
from urllib.parse import urljoin

os.makedirs("outputs", exist_ok=True)

BASE_URL = "https://sigaa.ufpb.br/sigaa/"
LOGIN_URL = urljoin(BASE_URL, "logon.jsf")

console.rule("Transporte HTTPS")

# Testar redirecionamento HTTP -> HTTPS
try:
    r_http = requests.get("http://sigaa.ufpb.br/sigaa/logon.jsf", allow_redirects=False, timeout=10)
    redirect_table = Table(title="Redirecionamento HTTP")
    redirect_table.add_column("Teste")
    redirect_table.add_column("Resultado")
    redirect_table.add_row("Status HTTP", str(r_http.status_code))
    redirect_table.add_row("Location", r_http.headers.get("Location", "N/A"))
    redirect_table.add_row("Redirecionamento", "Sim" if r_http.status_code in [301, 302, 307, 308] else "Não")
    console.print(redirect_table)
except Exception as e:
    rprint(f"[red]Erro no teste HTTP:[/red] {e}")

# Testar HTTPS e headers de segurança
try:
    r_https = requests.get(LOGIN_URL, timeout=10)
    
    security_table = Table(title="Headers de Segurança")
    security_table.add_column("Header")
    security_table.add_column("Valor")
    security_table.add_column("Status")
    
    headers_check = [
        ("strict-transport-security", "HSTS"),
        ("x-frame-options", "Clickjacking"),
        ("x-content-type-options", "MIME Sniffing"),
        ("content-security-policy", "CSP"),
        ("server", "Banner"),
        ("x-powered-by", "Fingerprint")
    ]
    
    for header, desc in headers_check:
        value = r_https.headers.get(header, "Ausente")
        status = "✅ Presente" if value != "Ausente" else "❌ Ausente"
        security_table.add_row(desc, value, status)
    
    console.print(security_table)
except Exception as e:
    rprint(f"[red]Erro no teste HTTPS:[/red] {e}")

──────────────────────────────────────────────── Transporte HTTPS ─────────────────────────────────────────────────

                   Redirecionamento HTTP                    
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Teste            ┃ Resultado                             ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status HTTP      │ 301                                   │
│ Location         │ https://sigaa.ufpb.br/sigaa/logon.jsf │
│ Redirecionamento │ Sim                                   │
└──────────────────┴───────────────────────────────────────┘

                             Headers de Segurança                             
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Header        ┃ Valor                                        ┃ Status      ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ HSTS          │ Ausente                                      │ ❌ Ausente  │
│ Clickjacking  │ sameorigin                                   │ ✅ Presente │
│ MIME Sniffing │ Ausente                                      │ ❌ Ausente  │
│ CSP           │ Ausente                                      │ ❌ Ausente  │
│ Banner        │ Apache-Coyote/1.1                            │ ✅ Presente │
│ Fingerprint   │ Servlet 2.5; JBoss-5.0/JBossWeb-2.1, JSF/1.2 │ ✅ Presente │
└───────────────┴──────────────────────────────────────────────┴─────────────┘

In [5]:
# Extração de ViewState e preparação de sessão
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) PentestBot/1.0"
})

console.rule("Análise JSF ViewState")

try:
    r1 = session.get(LOGIN_URL, timeout=20)
    
    # Extrair ViewState
    viewstate_pattern = r'name="javax\.faces\.ViewState"\s+value="([^"]+)"'
    viewstate_match = re.search(viewstate_pattern, r1.text)
    viewstate = viewstate_match.group(1) if viewstate_match else None
    
    vs_table = Table(title="ViewState JSF")
    vs_table.add_column("Atributo")
    vs_table.add_column("Valor")
    vs_table.add_row("Encontrado", "Sim" if viewstate else "Não")
    vs_table.add_row("Tamanho", str(len(viewstate)) if viewstate else "0")
    vs_table.add_row("Tipo", "Criptografado" if viewstate and len(viewstate) > 50 else "Simples")
    console.print(vs_table)
    
    # Análise de cookies iniciais
    cookie_table = Table(title="Cookies Iniciais")
    cookie_table.add_column("Nome")
    cookie_table.add_column("Valor")
    cookie_table.add_column("Flags")
    
    for cookie in session.cookies:
        flags = []
        if hasattr(cookie, 'secure') and cookie.secure:
            flags.append("Secure")
        if hasattr(cookie, '_rest') and cookie._rest.get('HttpOnly'):
            flags.append("HttpOnly")
        cookie_table.add_row(cookie.name, cookie.value[:20] + "..." if len(cookie.value) > 20 else cookie.value, ", ".join(flags) or "Nenhuma")
    
    console.print(cookie_table)
    
except Exception as e:
    rprint(f"[red]Erro na extração ViewState:[/red] {e}")
    viewstate = None

────────────────────────────────────────────── Análise JSF ViewState ──────────────────────────────────────────────

     ViewState JSF      
┏━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Atributo   ┃ Valor   ┃
┡━━━━━━━━━━━━╇━━━━━━━━━┩
│ Encontrado │ Não     │
│ Tamanho    │ 0       │
│ Tipo       │ Simples │
└────────────┴─────────┘

           Cookies Iniciais           
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Nome   ┃ Valor            ┃ Flags  ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ _75851 │ c88a054d6717228c │ Secure │
└────────┴──────────────────┴────────┘

In [7]:
# Teste de autenticação autorizado
console.rule("Autenticação SIGAA")

username = os.getenv("SIGAA_USER")
password = os.getenv("SIGAA_PASS")

if not username or not password:
    rprint("[red]ERRO:[/red] Credenciais não encontradas no .env")
else:
    rprint(f"[cyan]Usuário:[/cyan] {username}")
    
    # Capturar JSESSIONID antes do login
    jsid_before = None
    for cookie in session.cookies:
        if cookie.name.upper() == "JSESSIONID":
            jsid_before = cookie.value
    
    # Payload de login
    login_data = {
        "form": "form",
        "form:login": username,
        "form:senha": password,
        "form:entrar": "Entrar",
        "javax.faces.ViewState": viewstate or ""
    }
    
    try:
        r2 = session.post(LOGIN_URL, data=login_data, allow_redirects=False, timeout=20)
        
        # Capturar JSESSIONID após login
        jsid_after = None
        for cookie in session.cookies:
            if cookie.name.upper() == "JSESSIONID":
                jsid_after = cookie.value
        
        login_table = Table(title="Resultado do Login")
        login_table.add_column("Parâmetro")
        login_table.add_column("Valor")
        login_table.add_column("Análise")
        
        status = r2.status_code
        location = r2.headers.get("Location", "")
        
        login_table.add_row("Status", str(status), "✅ Sucesso" if status == 302 else "❌ Falha")
        login_table.add_row("Location", location, "✅ Redirecionamento" if location else "❌ Sem redirect")
        login_table.add_row("JSID Antes", jsid_before[:10] + "..." if jsid_before else "N/A", "")
        login_table.add_row("JSID Depois", jsid_after[:10] + "..." if jsid_after else "N/A", "")
        login_table.add_row("Rotação", "Sim" if jsid_before != jsid_after else "Não", "✅ Seguro" if jsid_before != jsid_after else "⚠️ Risco")
        
        console.print(login_table)
        
        # Testar acesso ao portal
        if location and "portal" in location.lower():
            portal_r = session.get(location if location.startswith("http") else urljoin(BASE_URL, location), timeout=20)
            rprint(f"[green]Portal acessível:[/green] Status {portal_r.status_code}")
        
    except Exception as e:
        rprint(f"[red]Erro no login:[/red] {e}")

─────────────────────────────────────────────── Autenticação SIGAA ────────────────────────────────────────────────

Usuário: guilhermemuniz

                        Resultado do Login                         
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Parâmetro   ┃ Valor                       ┃ Análise             ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ Status      │ 302                         │ ✅ Sucesso          │
│ Location    │ http://sigaa.ufpb.br/sigaa/ │ ✅ Redirecionamento │
│ JSID Antes  │ N/A                         │                     │
│ JSID Depois │ N/A                         │                     │
│ Rotação     │ Não                         │ ⚠️ Risco             │
└─────────────┴─────────────────────────────┴─────────────────────┘

In [8]:
# Probe controlado de brute force
import time
from tqdm import tqdm

console.rule("Teste Controlado Anti-Brute Force")

if username:
    BAD_PASSWORDS = ["123456", "admin", "teste", "senha123", "password"]
    MAX_ATTEMPTS = 5  # Atenção ao sobrecarregamento do sistema
    DELAY = 3  # segundos
    
    probe_results = []
    
    rprint(f"[yellow]Executando {MAX_ATTEMPTS} tentativas com senhas inválidas (delay: {DELAY}s)[/yellow]")
    
    for i, bad_pwd in enumerate(tqdm(BAD_PASSWORDS[:MAX_ATTEMPTS], desc="Testando"), 1):
        try:
            # Nova sessão para cada tentativa
            test_session = requests.Session()
            test_session.headers.update({"User-Agent": "Mozilla/5.0 PentestProbe/1.0"})
            
            # Obter novo ViewState
            r_fresh = test_session.get(LOGIN_URL, timeout=10)
            vs_match = re.search(r'name="javax\.faces\.ViewState"\s+value="([^"]+)"', r_fresh.text)
            fresh_vs = vs_match.group(1) if vs_match else ""
            
            # Tentativa com senha errada
            bad_data = {
                "form": "form",
                "form:login": username,
                "form:senha": bad_pwd,
                "form:entrar": "Entrar",
                "javax.faces.ViewState": fresh_vs
            }
            
            r_bad = test_session.post(LOGIN_URL, data=bad_data, allow_redirects=False, timeout=10)
            
            result = {
                "tentativa": i,
                "senha": bad_pwd,
                "status": r_bad.status_code,
                "location": r_bad.headers.get("Location", ""),
                "tamanho": len(r_bad.text)
            }
            probe_results.append(result)
            
            time.sleep(DELAY)
            
        except Exception as e:
            rprint(f"[red]Erro na tentativa {i}:[/red] {e}")
    
    probe_table = Table(title="Resultados do Probe")
    probe_table.add_column("#")
    probe_table.add_column("Senha")
    probe_table.add_column("Status")
    probe_table.add_column("Response Size")
    probe_table.add_column("Análise")
    
    for result in probe_results:
        analysis = "Bloqueado" if result["status"] == 429 else "Rejeitado" if result["status"] in [401, 403] else "Normal"
        probe_table.add_row(
            str(result["tentativa"]),
            result["senha"],
            str(result["status"]),
            str(result["tamanho"]),
            analysis
        )
    
    console.print(probe_table)
    
    # uniformidade
    sizes = [r["tamanho"] for r in probe_results]
    uniform = len(set(sizes)) == 1
    rprint(f"[cyan]Uniformidade das respostas:[/cyan] {'Sim' if uniform else 'Não'} - {'Seguro' if uniform else 'Possível enumeração'}")

──────────────────────────────────────── Teste Controlado Anti-Brute Force ────────────────────────────────────────

Executando 5 tentativas com senhas inválidas (delay: 3s)

Testando: 100%|██████████| 5/5 [01:07<00:00, 13.49s/it]



                Resultados do Probe                
┏━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ # ┃ Senha    ┃ Status ┃ Response Size ┃ Análise ┃
┡━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ 1 │ 123456   │ 302    │ 0             │ Normal  │
│ 2 │ admin    │ 302    │ 0             │ Normal  │
│ 3 │ teste    │ 302    │ 0             │ Normal  │
│ 4 │ senha123 │ 302    │ 0             │ Normal  │
│ 5 │ password │ 302    │ 0             │ Normal  │
└───┴──────────┴────────┴───────────────┴─────────┘

Uniformidade das respostas: Sim - Seguro

In [10]:
import json
from datetime import datetime
from pathlib import Path

console.rule("Relatório Final")

# Compilar achados
findings = {
    "timestamp": datetime.now().isoformat(),
    "target": BASE_URL,
    "user": username,
    "transport": {
        "https_redirect": r_http.status_code in [301, 302, 307, 308] if 'r_http' in locals() else "N/A",
        "hsts_present": "strict-transport-security" in r_https.headers if 'r_https' in locals() else False
    },
    "authentication": {
        "viewstate_present": viewstate is not None,
        "session_rotation": jsid_before != jsid_after if 'jsid_before' in locals() and 'jsid_after' in locals() else "N/A",
        "login_successful": status == 302 if 'status' in locals() else "N/A"
    },
    "security_headers": {
        header: r_https.headers.get(header, "Ausente") for header in [
            "strict-transport-security", "x-frame-options", 
            "x-content-type-options", "content-security-policy", "server", "x-powered-by"
        ]
    } if 'r_https' in locals() else {},
    "brute_force": {
        "attempts": len(probe_results) if 'probe_results' in locals() else 0,
        "uniform_responses": uniform if 'uniform' in locals() else "N/A"
    }
}

report_path = os.path.join("outputs", "pentest_report.json")
with open(report_path, "w", encoding="utf-8") as f:
    json.dump(findings, f, indent=2, ensure_ascii=False)

summary_table = Table(title="Resumo de Segurança")
summary_table.add_column("Categoria")
summary_table.add_column("Status")
summary_table.add_column("Observação")

categories = [
    ("Transporte HTTPS", "✅ OK" if findings["transport"].get("https_redirect") else "❌ Risco", "Redirecionamento automático"),
    ("HSTS", "✅ OK" if findings["transport"].get("hsts_present") else "❌ Ausente", "Strict-Transport-Security"),
    ("ViewState JSF", "✅ OK" if findings["authentication"].get("viewstate_present") else "❌ Ausente", "Token CSRF básico"),
    ("Rotação Sessão", "✅ OK" if findings["authentication"].get("session_rotation") else "⚠️ Risco", "Mitigação session fixation"),
    ("Anti-Brute Force", "✅ OK" if findings["brute_force"].get("uniform_responses") else "⚠️ Check", "Uniformidade de respostas")
]

for category, status, obs in categories:
    summary_table.add_row(category, status, obs)

console.print(summary_table)

rprint(f"[green]Relatório salvo em:[/green] {report_path}")
rprint(f"[cyan]Total de arquivos em outputs:[/cyan] {len(list(Path('outputs').glob('*')))} arquivos")

───────────────────────────────────────────────── Relatório Final ─────────────────────────────────────────────────

                      Resumo de Segurança                      
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Categoria        ┃ Status     ┃ Observação                  ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Transporte HTTPS │ ✅ OK      │ Redirecionamento automático │
│ HSTS             │ ❌ Ausente │ Strict-Transport-Security   │
│ ViewState JSF    │ ❌ Ausente │ Token CSRF básico           │
│ Rotação Sessão   │ ⚠️ Risco    │ Mitigação session fixation  │
│ Anti-Brute Force │ ✅ OK      │ Uniformidade de respostas   │
└──────────────────┴────────────┴─────────────────────────────┘

Relatório salvo em: outputs\pentest_report.json

Total de arquivos em outputs: 1 arquivos